In [24]:
"""
Вы работаете системным администратором и вам необходимо проанализировать логи
 доступа к веб-серверу. Логи хранятся в текстовом файле, где каждая строка
 представляет собой один запрос.
"""
import re # для парсинга
import json # для вывода
from collections import Counter # для подсчетов

# 2. Парсит каждую строку лога
def parse_log_line(line):
    pattern = r'^(\S+) - - \[.*?\] "(?P<method>\S+)\s(?P<url>\S+)\s\S+" (?P<status>\d{3}) \d+'
    match = re.match(pattern, line)
    if match:
        ip = match.group(1)
        method = match.group('method')
        url = match.group('url')
        status = match.group('status')
        return ip, method, url, status
    else:
        return None

# 1. Читает данные из файла логов
def read_log_file():
    filename = input("Введите имя файла логов (например, access.log): ").strip()
    try:
        parsed_data = []
        with open(filename, 'r') as f:
            for line in f:
                result = parse_log_line(line)
                if result:
                    parsed_data.append(result)
                else:
                    print(f'Предупреждение: некорректная строка — {line.strip()}')
            return parsed_data
    except FileNotFoundError:
        print(f"[!] Файл '{filename}' не найден.")
        return None

# 3. Собирает статистику
def calculate_statistics(data):
    ip_counter = Counter()
    method_counter = Counter()
    url_counter = Counter()
    status_counter = Counter()

    for ip, method, url, status in data:
        ip_counter[ip] += 1
        method_counter[method] += 1
        url_counter[url] += 1
        status_counter[status] += 1

    return {
        'requests_per_ip': ip_counter,
        'top_urls': url_counter.most_common(5),
        'requests_per_method': method_counter,
        'requests_per_status_code': status_counter
    }

# 4. Выводит собранную статистику
def print_statistics(stats):
    print("\n--- Log Analysis Report ---")
    print("\nRequests per IP:")
    for ip, count in stats['requests_per_ip'].items():
        print(f"  {ip}: {count}")

    print("\nTop 5 Requested URLs:")
    for resource, count in stats['top_urls']:
        print(f"  {resource}: {count}")

    print("\nRequests per HTTP Method:")
    for method, count in stats['requests_per_method'].items():
        print(f"  {method}: {count}")

    print("\nRequests per Status Code:")
    for status, count in stats['requests_per_status_code'].items():
        print(f"  {status}: {count}")

# 5. Записывает результаты анализа в JSON-файл
def save_to_json(stats, filename='report.json'):
    json_stats = {
        'requests_per_ip': dict(stats['requests_per_ip']),
        'top_urls': stats['top_urls'],
        'requests_per_method': dict(stats['requests_per_method']),
        'requests_per_status_code': dict(stats['requests_per_status_code'])
    }
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(json_stats, f, ensure_ascii=False, indent=4)
    files.download(filename)


# Основной блок
data = read_log_file()
stats = calculate_statistics(data)
print_statistics(stats)

save = input("\nСохранить отчет в JSON-файл? (y/n): ").lower()
if save == 'y':
    save_to_json(stats)
    print("(Optional) Report generated: report.json")


Введите имя файла логов (например, access.log): access.log
Предупреждение: некорректная строка — INVALID LOG LINE HERE

--- Log Analysis Report ---

Requests per IP:
  192.168.1.101: 3
  10.0.0.5: 2
  172.16.0.30: 1

Top 5 Requested URLs:
  /index.html: 3
  /api/submit: 1
  /style.css: 1
  /nonexistent.jpg: 1

Requests per HTTP Method:
  GET: 5
  POST: 1

Requests per Status Code:
  200: 4
  201: 1
  404: 1

Сохранить отчет в JSON-файл? (y/n): y


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(Optional) Report generated: report.json


In [23]:
"""
Вариант 3: Анализатор Ошибок
•	Задача:
1. Извлечь: IP-адрес, Ресурс, Код ответа.
2. Собрать статистику, фокусируясь на ошибках (коды 4xx и 5xx):
	Общее количество ошибок (4xx и 5xx).
	Топ-5 IP-адресов, с которых пришло больше всего запросов с ошибками.
	Топ-5 ресурсов (URL), которые чаще всего возвращали ошибку.
	Распределение кодов ошибок (например, 404: 50 раз, 500: 10 раз).
3. Вывести статистику в консоль.
"""
import re # для парсинга
from collections import Counter # для подсчетов

# 1. Извлечь: IP-адрес, Ресурс, Код ответа
def parse_log_line(line):
    pattern = r'^(\S+) - - \[.*?\] "(?P<method>\S+)\s(?P<url>\S+)\s\S+" (?P<status>\d{3}) \d+'
    match = re.match(pattern, line)
    if match:
        ip = match.group(1)
        url = match.group('url')
        status = match.group('status')
        return ip, url, int(status)
    else:
        return None

# Читает файл и собирает только ошибочные строки
def read_error_data():
    filename = input("Введите имя файла логов (например, access.log): ").strip()
    try:
        error_data = []
        with open(filename, 'r') as f:
            for line in f:
                parsed = parse_log_line(line)
                if parsed:
                    ip, url, status = parsed
                    if 400 <= status < 600:
                        error_data.append((ip, url, status))
                else:
                    print(f'Предупреждение: некорректная строка — {line.strip()}')
            return error_data
    except FileNotFoundError:
        print(f"[!] Файл '{filename}' не найден.")
        return None

# 2. Собрать статистику, фокусируясь на ошибках (коды 4xx и 5xx)
def calculate_statistics(data):
    total_errors = len(data)
    ip_counter = Counter()
    url_counter = Counter()
    status_counter = Counter()

    for ip, url, status in data:
        ip_counter[ip] += 1
        url_counter[url] += 1
        status_counter[status] += 1

    return {
        'total_errors': total_errors,
        'top_ips': ip_counter.most_common(5),
        'top_urls': url_counter.most_common(5),
        'status_distribution': dict(status_counter)
    }

# 3. Вывести статистику в консоль.
def print_statistics(stats):
    print("\n--- Error Analysis Report ---")

    print("\nTotal Number of Errors:")
    print(f"  {stats['total_errors']}")

    print("\nTop 5 IPs with Most Errors:")
    for ip, count in stats['top_ips']:
        print(f"  {ip}: {count}")

    print("\nTop 5 Resources Causing Errors:")
    for url, count in stats['top_urls']:
        print(f"  {url}: {count}")

    print("\nError Code Distribution:")
    for code, count in stats['status_distribution'].items():
        print(f"  {code}: {count}")


# Основной блок
data = read_error_data()
stats = calculate_statistics(data)
print_statistics(stats)


Введите имя файла логов (например, access.log): access.log
Предупреждение: некорректная строка — INVALID LOG LINE HERE

--- Error Analysis Report ---

Total Number of Errors:
  1

Top 5 IPs with Most Errors:
  172.16.0.30: 1

Top 5 Resources Causing Errors:
  /nonexistent.jpg: 1

Error Code Distribution:
  404: 1
